In [1]:
import config
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from googletrans import Translator
from tweepyclient import TweepyClient

In [2]:
bearer_token = config.BEARER_TOKEN
tweepyClient = TweepyClient(bearer_token)
tweepyClient

In [50]:
def format_for_csv(text):
    return text.replace(',',' ').replace("\n",' ') #Data must be cleaned for the CSV to be readed clearly 


def df_from_tweets(tweet_list, data=[], columns=['id', 'tweet', 'entidad', 'dominio']):
    for tweet in tweet_list:
        entity = ''
        domain = ''
        for context_annotation in tweet.context_annotations:
            entity = context_annotation['entity']['name']
            domain = context_annotation['domain']['name']
        cleaned_text = format_for_csv(tweet.text)
        data.append([tweet.id, cleaned_text, entity, domain])
    return pd.DataFrame(data, columns=columns)

def translate(word, dest='en', translator = Translator()):
    translation = translator.translate(word, dest)
    return translation.text

def get_tweets_sentiment(tweet_list, data=[], columns=['id', 'tweet', 'sentimiento', 'entidad'], sia = SentimentIntensityAnalyzer()):
    for tweet in tweet_list:
        entity = ''
        translated_text = translate(tweet.text)  #Sentiment analysis only works in english
        for context_annotation in tweet.context_annotations:
            entity = context_annotation['entity']['name']
        # SENTIMENT ONLY WORKS IN ENGLISH
        sentiment_dict = sia.polarity_scores(translated_text)
        positive = sentiment_dict["pos"]
        negative = sentiment_dict["neg"]
        bias = 'neutral'
        if (positive > negative):
            bias = 'positivo'
        elif(positive < negative):
            bias = 'negativo'
        else:
            bias = 'neutral'
        cleaned_text = format_for_csv(tweet.text)
        data.append([tweet.id, cleaned_text, bias, entity])
    # SENTIMENT ANALYSIS DATAFRAME
    return pd.DataFrame(data, columns=columns)

Pregunta #1: Analisis de sentimientos sobre el tema vacunas

In [ ]:
query = "vacuna (context:131.1220701888179359745 OR context:123.1220701888179359745) -is:retweet lang:es"
vaccine_tweets = tweepyClient.search_tweets(query, 10000)

In [ ]:
vaccine_tweets_df = get_tweets_sentiment(vaccine_tweets)
vaccine_tweets_df.to_csv('./data/sentimientos_vacuna2.csv')

Pregunta #2: Sintomas más consultados

In [ ]:
query = "(sintomas OR sintoma) context:123.1220701888179359745 lang:es -is:retweet"
symptom_tweets = tweepyClient.search_tweets(query, 10000)

In [ ]:
sp_tweets_df = df_from_tweets(symptom_tweets)
sp_tweets_df.to_csv('./data/symptom_tweets.csv')

Pregunta #3: Temas más consultados sobre el covid en twitter

In [32]:
query = "context:123.1220701888179359745 lang:es -is:retweet"
covid_tweets = tweepyClient.search_tweets(query, 10000)

In [33]:
covid_tweets_df = df_from_tweets(covid_tweets)
sp_tweets_df.to_csv('./data/covid_tweets.csv')

Pregunta #4

In [42]:
query = "(dioxido OR azitromicina OR ivermectina) context:123.1220701888179359745 -is:retweet"
medicine_tweets = tweepyClient.search_tweets(query, 10000)

In [ ]:
sp_tweets_df = get_tweets_sentiment(medicine_tweets)
sp_tweets_df.to_csv('./data/4medicina_alternativa.csv')

Pregunta #5: Cuales fueron los hashtags más usados sobre el COVID-19

In [54]:
query = "(context:123.1220701888179359745 OR context:131.1220701888179359745) -is:retweet has:hashtags lang:es"
tweets_with_hashtag = tweepyClient.search_tweets(query, 40000)

In [56]:
hashtag_tweets_df = df_from_tweets(tweets_with_hashtag)
hashtag_tweets_df.to_csv('./data/5hashtag_tweets.csv')